In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
import pickle

In [2]:
# load dataset
data=pd.read_csv('Churn_Modelling.csv')
data.head(5)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
# drop unnecessary columns
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [4]:
# encore one categorical variable
label_encoder_gender = LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])
data.head(10)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
5,645,Spain,1,44,8,113755.78,2,1,0,149756.71,1
6,822,France,1,50,7,0.00,2,1,1,10062.80,0
7,376,Germany,0,29,4,115046.74,4,1,0,119346.88,1
8,501,France,1,44,4,142051.07,2,0,1,74940.50,0
9,684,France,1,27,2,134603.88,1,1,1,71725.73,0


In [5]:
# one hot encode for geography column

one_hot_en=OneHotEncoder()
geo_encoded=one_hot_en.fit_transform(data[['Geography']]).toarray()
geo_encoded

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [6]:
one_hot_en.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [8]:
geo_encoded_pd=pd.DataFrame(geo_encoded,columns=one_hot_en.get_feature_names_out(['Geography']))

In [9]:
## combine all the columns with the one_hot_encoded columns

data=pd.concat([data,geo_encoded_pd],axis=1)
data=data.drop(['Geography'],axis=1)
data.head(10)

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
5,645,1,44,8,113755.78,2,1,0,149756.71,1,0.0,0.0,1.0
6,822,1,50,7,0.00,2,1,1,10062.80,0,1.0,0.0,0.0
7,376,0,29,4,115046.74,4,1,0,119346.88,1,0.0,1.0,0.0
8,501,1,44,4,142051.07,2,0,1,74940.50,0,1.0,0.0,0.0
9,684,1,27,2,134603.88,1,1,1,71725.73,0,1.0,0.0,0.0


In [10]:
## save the encoders and sscaler

with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)
    
with open('one_hot_en.pkl', 'wb') as file:
    pickle.dump(one_hot_en, file)

In [11]:
# devide the dataset in independent and dependent features

X=data.drop('Exited',axis=1)
y=data['Exited']


# split it into train and test datasets
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=36)

# scale thease feature

scaler=StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [12]:
with open ('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [13]:
import tensorflow as tf
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [14]:
## build the ann model (ann model)

model = Sequential(
    [
        Dense(units=64, activation='relu', input_shape=(X_train.shape[1],)), # HL1 input layer
        Dense(units=32, activation='relu'), # HL2
        Dense(units=1, activation='sigmoid') # output layer with sigmoid activation
    ]
)


In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:
## compile the model
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=['accuracy'])

In [17]:
## set up the tensorboard 
log_dir='logs/fit' + datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [18]:
## set up early stopping 
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [19]:
#training the model
history = model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100,callbacks=[tensorflow_callback,early_stopping_callback])

Epoch 1/100


250/250 [==============================] - 8s 13ms/step - loss: 0.4398 - accuracy: 0.8117 - val_loss: 0.4390 - val_accuracy: 0.8080
Epoch 2/100
250/250 [==============================] - 2s 7ms/step - loss: 0.3671 - accuracy: 0.8526 - val_loss: 0.4110 - val_accuracy: 0.8255
Epoch 3/100
250/250 [==============================] - 2s 7ms/step - loss: 0.3407 - accuracy: 0.8633 - val_loss: 0.3995 - val_accuracy: 0.8310
Epoch 4/100
250/250 [==============================] - 2s 9ms/step - loss: 0.3311 - accuracy: 0.8668 - val_loss: 0.4004 - val_accuracy: 0.8260
Epoch 5/100
250/250 [==============================] - 2s 7ms/step - loss: 0.3261 - accuracy: 0.8685 - val_loss: 0.3918 - val_accuracy: 0.8290
Epoch 6/100
250/250 [==============================] - 2s 7ms/step - loss: 0.3221 - accuracy: 0.8716 - val_loss: 0.4046 - val_accuracy: 0.8280
Epoch 7/100
250/250 [==============================] - 2s 6ms/step - loss: 0.3192 - accuracy: 0.8723 - val_loss: 0.3954 - val_accuracy: 0.8

In [20]:
model.save('annmodel.h5')

c:\Users\city\Generative AI with LANGCHAIN-KN\ANN-PROJECT\ann-sample\venv_ann\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [21]:
## load tensorboard extension
%load_ext tensorboard 

In [22]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6007 (pid 14372), started 1 day, 18:18:53 ago. (Use '!kill 14372' to kill it.)